In [1]:
pwd

'/home/wsuser/work'

In [2]:
from keras.preprocessing.image import ImageDataGenerator

In [3]:
#image Data Agumentation
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)


In [4]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='1V9GxTxF5n3PSt7I1lZ2azQsVxsJf04ximYOwQu1icoA',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'radiologyimages-donotdelete-pr-hxwo7rh2vej7in'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [6]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [7]:
pwd

'/home/wsuser/work'

In [8]:
import os
import cv2
import numpy as np

In [9]:
datapath =r"/home/wsuser/work/Dataset/train"

In [10]:
#Loading our data and performing data agumentation
x_train = test_datagen.flow_from_directory(r"/home/wsuser/work/Dataset/train",target_size=(64, 64),batch_size=5,color_mode='grayscale',class_mode='categorical')

Found 594 images belonging to 6 classes.


In [11]:
datapath =r"/home/wsuser/work/Dataset/test"

In [12]:
x_test = test_datagen.flow_from_directory(r"/home/wsuser/work/Dataset/test",target_size=(64, 64),batch_size=5,color_mode='grayscale',class_mode='categorical')

Found 30 images belonging to 6 classes.


In [13]:
#importing Necessary Libraries
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [14]:
#Initializing the model
model=Sequential()

In [15]:
#First convolution Layer and pooling
model.add(Conv2D(32,(3,3),input_shape=(64, 64, 1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [16]:
#Second convolution Layer and pooling
model.add(Conv2D(32,(3, 3),activation='relu'))
#input_shape is going to be the pooled feature mapsfrom the previous convolution layer
model.add(MaxPooling2D(pool_size=(2, 2)))

In [17]:
#Flattening the layers
model.add(Flatten())

In [18]:
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=6,activation='softmax'))

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [20]:
#compiling the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [21]:
#Fitting the model
model.fit_generator(
    generator=x_train,steps_per_epoch = len(x_train),
    epochs=20, validation_data=x_test,validation_steps = len(x_test))

Epoch 1/20


/tmp/wsuser/ipykernel_4265/1851565379.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


119/119 [==============================] - 5s 39ms/step - loss: 1.3425 - accuracy: 0.4916 - val_loss: 0.7397 - val_accuracy: 0.6667
Epoch 2/20
119/119 [==============================] - 5s 38ms/step - loss: 0.4886 - accuracy: 0.8350 - val_loss: 0.5630 - val_accuracy: 0.8667
Epoch 3/20
119/119 [==============================] - 5s 38ms/step - loss: 0.2364 - accuracy: 0.9343 - val_loss: 0.4040 - val_accuracy: 0.9000
Epoch 4/20
119/119 [==============================] - 5s 38ms/step - loss: 0.0861 - accuracy: 0.9731 - val_loss: 0.5066 - val_accuracy: 0.8667
Epoch 5/20
119/119 [==============================] - 5s 38ms/step - loss: 0.0419 - accuracy: 0.9916 - val_loss: 0.5657 - val_accuracy: 0.9000
Epoch 6/20
119/119 [==============================] - 5s 38ms/step - loss: 0.0080 - accuracy: 1.0000 - val_loss: 0.5177 - val_accuracy: 0.9333
Epoch 7/20
119/119 [==============================] - 4s 38ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.5746 - val_accuracy: 0.9333
Epoch 8/20

In [22]:
#Save the model
model.save('gesture.h5')

In [23]:
!tar -zcvf image-classification-model_new.tgz gesture.h5

gesture.h5


In [24]:
ls-1

Dataset/
gesture.h5
image-classification-model_new.tgz
model-bw.json


In [25]:
!pip install watson-machine-learning-client --upgrade

In [26]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                     "apikey": "w5JniNEQHFTew62fJhNWzBJ8GNy8oYZrjGGeQrltphQl"
                   }
client = APIClient(wml_credentials)

In [27]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [29]:
space_uid = guid_from_space_name(client,'MODEL')
print("Space UID = " + space_uid)

Space UID = 130aa374-85e6-47f0-b1b5-e823c4488f5e


In [30]:
client.set.default_space(space_uid)

'SUCCESS'

In [31]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [32]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [33]:
model_details = client.repository.store_model(model= 'image-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN-model",
    client.repository.ModelMetaNames.TYPE:"tensorflow_rt22.1",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                             )
model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [34]:
model_id

'f7b2e660-c043-465b-a7b0-63aec4f6a2e7'

In [35]:
client.repository.download(model_id, 'my_model_1.tar.gz')

Successfully saved model content to file: 'my_model_1.tar.gz'


'/home/wsuser/work/my_model_1.tar.gz'

In [36]:
model_json=model.to_json()
with open("model-bw.json","w")as json_file:
    json_file.write(model_json)

In [37]:
from tensorflow.keras.models import load_model

In [38]:
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.applications.inception_v3 import preprocess_input
model=load_model("gesture.h5")

In [39]:
img=image.load_img(r"/home/wsuser/work/Dataset/test/1/1.jpg",grayscale=True,target_size=(64,64))

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [40]:
x=image.img_to_array(img)

In [41]:
import numpy as np

In [42]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 64, 64, 1)

In [43]:
pred=np.argmax(model.predict(x))

In [44]:
pred

1

In [45]:
model.predict(x)


array([[1.00672705e-05, 9.49776113e-01, 5.02128787e-02, 7.69676545e-11,
        6.78264438e-16, 8.97140580e-07]], dtype=float32)